In [1]:
#require "nomad";;
open Nomad
open Nomad.Expr
open Nomad.Simplify
open Nomad.Eval
open Nomad.Derivatives
open Nomad.Gradient_descent
open Nomad.Newton
open Nomad.Bisection

let x = Var "x"
let y = Var "y"
let z = Var "z"

val x : Nomad.Expr.expr = Var "x"


val y : Nomad.Expr.expr = Var "y"


val z : Nomad.Expr.expr = Var "z"


# Basic symbolic computations

In [2]:
let f = (x ^: Float 2.) +: (y ^: Float 3.);;
let env: env = [("x", 1.); ("y", 2.)];;
eval env f;;

val f : Nomad.Expr.expr =
  Sum [Pow (Var "x", Float 2.); Pow (Var "y", Float 3.)]


val env : Nomad.Eval.env = [("x", 1.); ("y", 2.)]


- : float = 9.


In [3]:
derivative "x" f |> string_of_expr;;
derivative "y" f |> string_of_expr;;

nth_derivative "y" 2 f |> string_of_expr;;
nth_derivative "y" 3 f |> string_of_expr;;
nth_derivative "y" 4 f |> string_of_expr;;

- : string = "2 * x"


- : string = "3 * y^2"


- : string = "6 * y"


- : string = "6"


- : string = "0"


In [5]:
let f = x ^: Float (-.1.);;
f |> string_of_expr;;

nth_derivative "x" 1 f |> string_of_expr;;
nth_derivative "x" 2 f |> string_of_expr;;
nth_derivative "x" 3 f |> string_of_expr;;
nth_derivative "x" 4 f |> string_of_expr;;

val f : Nomad.Expr.expr = Pow (Var "x", Float (-1.))


- : string = "x^{-1}"


- : string = "-1 * x^{-2}"


- : string = "2 * x^{-3}"


- : string = "-6 * x^{-4}"


- : string = "24 * x^{-5}"


In [6]:
(* Zadanie z AN - lista 13 zad. 3 *)
let e = Cos ((Float 3. *: x) +: (pi /: Float 4.)) in
let e' = nth_derivative "x" 4 e in
string_of_expr e';;

- : string = "81 * cos(0.785398163397 + 3 * x)"


# Simplify

In [7]:
let f = Neg(x) +: Neg(y) +: z +: x +: y;;
(* kinda ugly AST *)

val f : Nomad.Expr.expr =
  Sum
   [Sum [Sum [Sum [Neg (Var "x"); Neg (Var "y")]; Var "z"]; Var "x"];
    Var "y"]


In [8]:
simplify f
(* now beautiful *)

- : Nomad.Expr.expr = Var "z"


## Jak to się dzieje?
1. Pokaż AST
2. Pokaż simplify z sympy
3. Pokaż simplify mojego +/-

TODO: tralalalalal

In [18]:
let g = x +: y +: z;;
let f = (Sin(g) *: Sin(g)) +: (Cos(g) *: Cos(g));; (* sin(g)^2 + cos(g)^2 *)

simplify f;;

val g : Nomad.Expr.expr = Sum [Sum [Var "x"; Var "y"]; Var "z"]


val f : Nomad.Expr.expr =
  Sum
   [Product
     [Sin (Sum [Sum [Var "x"; Var "y"]; Var "z"]);
      Sin (Sum [Sum [Var "x"; Var "y"]; Var "z"])];
    Product
     [Cos (Sum [Sum [Var "x"; Var "y"]; Var "z"]);
      Cos (Sum [Sum [Var "x"; Var "y"]; Var "z"])]]


- : Nomad.Expr.expr = Float 1.


In [24]:
let f = ((x ^: Float 3.) *: (x ^: Float 39.) *: (x ^: Float 0.42)) +: (y ^: Float 1.23) /: (y ^: Float 0.23);;
let s = simplify f;;
s |> string_of_expr;;

val f : Nomad.Expr.expr =
  Sum
   [Product
     [Product [Pow (Var "x", Float 3.); Pow (Var "x", Float 39.)];
      Pow (Var "x", Float 0.42)];
    Product
     [Pow (Var "y", Float 1.23);
      Pow (Pow (Var "y", Float 0.23), Float (-1.))]]


val s : Nomad.Expr.expr = Sum [Pow (Var "x", Float 42.42); Var "y"]


- : string = "x^42.42 + y"


In [35]:
let f = Exp(Float 1.);;
f |> simplify |> string_of_expr;;

let f = Float 3.14159265358979323846;; (* FIXME: *)
f |> string_of_expr;;

let f = Exp(x) ^: Float 2.;;
f |> simplify |> string_of_expr;;

val f : Nomad.Expr.expr = Exp (Float 1.)


- : string = "e"


val f : Nomad.Expr.expr = Float 3.14159265358979312


- : string = "π"


val f : Nomad.Expr.expr = Pow (Exp (Var "x"), Float 2.)


- : string = "exp(2. * x)"


# Numerical Optimization

### Newton

In [45]:
let eq: equation = (x, Float 2.);;
solve_newton eq ~initial_guess:0.1 ~max_iter:10;;

val eq : Nomad.Expr.equation = (Var "x", Float 2.)


- : (float, Nomad.Optimization_types.optim_error) result = Ok 2.


In [46]:
let eq: equation = (x *: x, Float 2.);;
solve_newton eq ~initial_guess:0.1 ~max_iter:10;;

val eq : Nomad.Expr.equation = (Product [Var "x"; Var "x"], Float 2.)


- : (float, Nomad.Optimization_types.optim_error) result =
Ok 1.41421360011580322


In [62]:
let eq: equation = (Sin(x), Cos(x));;
solve_newton eq ~initial_guess:0. ~max_iter:10;;

val eq : Nomad.Expr.equation = (Sin (Var "x"), Cos (Var "x"))


- : (float, Nomad.Optimization_types.optim_error) result =
Ok 0.785398175999701853


In [79]:
let eq: equation = (x ^: Float 3., Float 8.);;
solve_newton eq ~initial_guess:0.1 ~max_iter:20;;

val eq : Nomad.Expr.equation = (Pow (Var "x", Float 3.), Float 8.)


- : (float, Nomad.Optimization_types.optim_error) result =
Ok 2.00000000000348122


In [94]:
let eq: equation = (Log(x), Float 1.);;
solve_newton eq ~initial_guess:0.1 ~max_iter:20;;

val eq : Nomad.Expr.equation = (Log (Var "x"), Float 1.)


- : (float, Nomad.Optimization_types.optim_error) result =
Ok 2.71828177920980218


In [117]:
let eq: equation = ((x^:Float 2.) /: (y ^: Float 1.5), Float 18.);;
solve_newton_multivar eq ~initial_guess:["x", 2.; "y", 0.5] ~max_iter:100;;

let f x y = (x *. x) /. (Float.pow y 1.5);;
f 2.04713834 0.378455;;

val eq : Nomad.Expr.equation =
  (Product
    [Pow (Var "x", Float 2.); Pow (Pow (Var "y", Float 1.5), Float (-1.))],
   Float 18.)


- : (Nomad.Eval.env, Nomad.Optimization_types.optim_error) result =
Ok [("x", 2.0471383398778249); ("y", 0.378455295124827507)]


val f : float -> float -> float = <fun>


- : float = 18.0000210571891195


### Bisekcja

In [121]:
(* val solve_bisection :
  Expr.equation ->
  a:float ->
  b:float ->
  tolerance:float ->
  max_iter:int ->
  (float, optim_error) result *)

let eq: equation = (x ^: Float 2., Float 2.);;
solve_bisection eq ~a:0. ~b:2. ~tolerance:0.000001 ~max_iter:100;;;;


val eq : Nomad.Expr.equation = (Pow (Var "x", Float 2.), Float 2.)


- : (float, Nomad.Optimization_types.optim_error) result =
Ok 1.41421365737915039


### Gradient descent

In [139]:
(* val solve_gradient_descent :
  Expr.equation ->
  initial_guess:Eval.env ->
  learning_rate:float ->
  max_iter:int ->
  (Eval.env, optim_error) result *)

let eq: equation = (x +: y, Float 2.);;
let initial_guess: env = [("x", -1.); ("y", -2.)];;
solve_gradient_descent eq ~initial_guess ~learning_rate:0.1 ~max_iter:100;

val eq : Nomad.Expr.equation = (Sum [Var "x"; Var "y"], Float 2.)


val initial_guess : Nomad.Eval.env = [("x", -1.); ("y", -2.)]


- : (Nomad.Eval.env, Nomad.Optimization_types.optim_error) result =
Ok [("x", 1.50000000000000044); ("y", 0.499999999999999445)]


In [140]:
let eq: equation = ((x *: y) +: (x *: x) +: (y *: y), Float 8.);;
let initial_guess: env = [("x", 1.); ("y", 0.)];;
solve_gradient_descent eq ~initial_guess ~learning_rate:0.01 ~max_iter:100;;

let f x y = (x *. y) +. (x *. x) +. (y *. y);;
f 2.33242 0.81364;;

val eq : Nomad.Expr.equation =
  (Sum
    [Sum [Product [Var "x"; Var "y"]; Product [Var "x"; Var "x"]];
     Product [Var "y"; Var "y"]],
   Float 8.)


val initial_guess : Nomad.Eval.env = [("x", 1.); ("y", 0.)]


- : (Nomad.Eval.env, Nomad.Optimization_types.optim_error) result =
Ok [("x", 2.33242880429706068); ("y", 0.813642134235871)]


val f : float -> float -> float = <fun>


- : float = 7.99994331479999943


# Reszta

In [89]:
derivative "x" (Log(x *: y)) |> simplify;; (* FIXME: *)

- : Nomad.Expr.expr =
Product [Var "y"; Pow (Var "x", Float (-1.)); Pow (Var "y", Float (-1.))]


In [ ]:
let e1 = (x *: (y ^: Float 2.)) in
let e2 = (y *: x) in

simplify (e1 /: e2) (* TODO: rozdzielac *)

- : Nomad.Expr.expr =
Product
 [Var "x"; Pow (Var "y", Float 2.);
  Pow (Product [Var "x"; Var "y"], Float (-1.))]


In [7]:
let e = (Float 1. *: y *: y) +: (Float 2. *: y *: Float 2.);;

simplify e |> string_of_expr;;

val e : Nomad.Expr.expr =
  Sum
   [Product [Product [Float 1.; Var "y"]; Var "y"];
    Product [Product [Float 2.; Var "y"]; Float 2.]]


- : string = "4. * y + y^2."


In [7]:
let e = (Var "x" ^: Float 3.) in
let e2 = (Var "x" ^: Float 2.) in
simplify (e /: e2) (* (x^a)^b = x^ab *)

- : Nomad.Expr.expr =
Product [Pow (Var "x", Float 3.); Pow (Pow (Var "x", Float 2.), Float (-1.))]


In [9]:
let e = Float 2. *: Var "y" *: Float 2.;;

e |> collect |> string_of_expr;;

val e : Nomad.Expr.expr = Product [Product [Float 2.; Var "y"]; Float 2.]


- : string = "4. * y"


In [10]:
let e = Product ([Float 1.; Var "y"; Float 2.; Var "x"; Float 3.;  Var "x"; Var "y"]) in 

simplify e |> string_of_expr;;

- : string = "6. * x^2. * y^2."


In [12]:
let e = Float 3. *: Var "z" /: Var "z";;

simplify e |> string_of_expr;;

val e : Nomad.Expr.expr =
  Product [Product [Float 3.; Var "z"]; Pow (Var "z", Float (-1.))]


- : string = "3."


In [4]:
(* Financial Mathematics - Black-Scholes Option Pricing *)
let black_scholes_example () =
  let s = Var "S" in        (* Stock price *)
  let k = Float 100.0 in    (* Strike price *)
  let r = Float 0.05 in     (* Risk-free rate *)
  let t = Var "t" in        (* Time to expiration *)
  let sigma = Float 0.2 in  (* Volatility *)
  
  (* Call option price formula (simplified) *)
  let d1 = (Log(s /: k) +: (r +: (sigma ^: Float 2.) /: Float 2.) *: t) /: 
           (sigma *: Pow(t, Float 0.5)) in
  
  let derivative = derivative d1 "S" |> simplify in
  
  print_endline "Black-Scholes Option Delta:";
  print_endline (string_of_expr derivative);
  print_endline "Value at S=100, t=1:";
  print_endline (string_of_float (eval [("S", 100.0); ("t", 1.0)] derivative))

  
let () = black_scholes_example ()

val black_scholes_example : unit -> unit = <fun>


Black-Scholes Option Delta:
0.01 / 0.01 * S / 0.2 * t^0.5 + -1. / 0.2 * t^0.5 * 0.2 * 0.5 / t * (0.07 * t + log(0.01 * S))
Value at S=100, t=1:
-0.125


In [5]:
(* Machine Learning - Neural Network Analysis *)
let neural_network_example () =
  (* ReLU activation function and its derivative *)
  let relu x = Max(x, Float 0.) in
  let x = Var "x" in
  let hidden_layer = [relu (x *: Float 2.); relu (x *: Float (-1.) +: Float 1.)] in
  let output = List.fold_left (+:) (Float 0.) hidden_layer in
  
  let gradient = derivative output "x" |> simplify in
  
  print_endline "Neural Network with ReLU Activation:";
  print_endline ("f(x) = " ^ latex_of_expr output);
  print_endline ("f'(x) = " ^ latex_of_expr gradient)

error: compile_error

In [36]:
(* Physics - Projectile Motion with Air Resistance *)
let projectile_motion_example () =
  let t = Var "t" in
  let v0 = Float 50.0 in  (* Initial velocity *)
  let theta = Float (3.14 /. 4.) in  (* Launch angle *)
  let g = Float 9.81 in
  let k = Float 0.1 in  (* Air resistance coefficient *)
  
  (* Position equations *)
  let x = v0 *: Cos(theta) *: t -: (k /: Float 2.) *: Pow(t, Float 2.) in
  let y = v0 *: Sin(theta) *: t -: (g /: Float 2.) *: Pow(t, Float 2.) in
  
  print_endline "Projectile Motion Equations:";
  print_endline ("x(t) = " ^ latex_of_expr x);
  print_endline ("y(t) = " ^ latex_of_expr y);
  
  (* Find time of landing *)
  let eq = (y, Float 0.) in
  match solve_newton eq ~initial_guess:5.0 ~max_iter:100 with
  | Ok t_land -> print_endline ("Landing time: " ^ string_of_float t_land)
  | Error msg -> print_endline msg

error: compile_error

In [37]:
(* Engineering - Beam Deflection Analysis *)
let beam_deflection_example () =
  let x = Var "x" in
  let L = Float 10.0 in  (* Beam length *)
  let EI = Float 2000.0 in  (* Flexural rigidity *)
  let w = Float 1.0 in  (* Distributed load *)
  
  (* Beam deflection equation *)
  let deflection = (w *: Pow(x, Float 4.)) /: (Float 24. *: EI) -:
                   (w *: L *: Pow(x, Float 3.)) /: (Float 12. *: EI) in
  
  let max_deflection = derivative deflection "x" |> simplify in
  
  print_endline "Beam Deflection Analysis:";
  print_endline ("y(x) = " ^ latex_of_expr deflection);
  print_endline "Maximum deflection occurs at:";
  
  (* Find point of maximum deflection *)
  let eq = (max_deflection, Float 0.) in
  match solve_newton eq ~initial_guess:5.0 ~max_iter:100 with
  | Ok x_max -> print_endline ("x = " ^ string_of_float x_max)
  | Error msg -> print_endline msg

error: compile_error

In [38]:
(* Run examples *)
let () =
  print_endline "\n=== Financial Mathematics ===\n";
  black_scholes_example ();
  
  print_endline "\n=== Machine Learning ===\n";
  neural_network_example ();
  
  print_endline "\n=== Physics ===\n";
  projectile_motion_example ();
  
  print_endline "\n=== Engineering ===\n";
  beam_deflection_example ()

error: compile_error

In [27]:
(* Example 1: Neural Network Activation Function Analysis *)
let sigmoid x = Div(Float 1., Float 1. +: Exp(Float (-1.) *: x)) in
let x = Var "x" in
let expr = sigmoid x in
let derivative_expr = derivative expr "x" |> simplify in

make_output "Neural Network Activation Function" (
  "Original sigmoid: " ^ string_of_expr expr ^ "\n" ^
  "Derivative: " ^ string_of_expr derivative_expr ^ "\n" ^
  "LaTeX: " ^ latex_of_expr derivative_expr
) |> print_endline;;



Neural Network Activation Function
Original sigmoid: 1. / (1. + exp(-1. * x))
Derivative: (0. - -1. * exp(-1. * x)) / ((exp(-1. * x) + 1.) * (exp(-1. * x) + 1.))
LaTeX: \frac{0. - -1. \cdot e^{-1. \cdot x}}{(e^{-1. \cdot x} + 1.) \cdot (e^{-1. \cdot x} + 1.)}



- : unit = ()
